## 回帰の入門 - レッスン1

#### 視点を広げてみよう

✅ 回帰にはさまざまな方法があり、どの方法を選ぶかは求めている答えによります。たとえば、特定の年齢の人の身長を予測したい場合は、**数値的な値**を求めているので `線形回帰` を使用します。一方、ある料理がビーガン料理と見なされるべきかどうかを知りたい場合は、**カテゴリの割り当て**を求めているので `ロジスティック回帰` を使用します。ロジスティック回帰については後ほど学びます。データに対してどのような質問ができるか、そしてこれらの方法のどれが適切かを少し考えてみてください。

このセクションでは、[糖尿病に関する小さなデータセット](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)を使用します。糖尿病患者に対する治療をテストしたいと想像してみてください。機械学習モデルは、変数の組み合わせに基づいて、どの患者が治療により良く反応するかを判断するのに役立つかもしれません。非常に基本的な回帰モデルであっても、視覚化することで、理論的な臨床試験を整理するのに役立つ変数に関する情報を示すことができます。

それでは、このタスクに取り組んでみましょう！

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>イラスト: @allison_horst</figcaption>

<!--![イラスト: \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.ja.jpg)<br>イラスト: @allison_horst-->


## 1. ツールセットの準備

このタスクでは、以下のパッケージが必要です：

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) は、データサイエンスをより速く、簡単で楽しいものにするために設計された[Rパッケージのコレクション](https://www.tidyverse.org/packages)です。

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) フレームワークは、モデリングと機械学習のための[パッケージのコレクション](https://www.tidymodels.org/packages/)です。

以下のコマンドでインストールできます：

`install.packages(c("tidyverse", "tidymodels"))`

以下のスクリプトは、このモジュールを完了するために必要なパッケージがインストールされているかを確認し、足りない場合は自動的にインストールします。


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



では、これらの素晴らしいパッケージを読み込み、現在のRセッションで利用可能にしましょう。（これは単なる例示のためであり、`pacman::p_load()`はすでにそれを行っています）


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. 糖尿病データセット

この演習では、糖尿病データセットを使って回帰スキルを活用し、予測を行います。[糖尿病データセット](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt)には、糖尿病に関する`442件のサンプル`が含まれており、10個の予測変数（`年齢`、`性別`、`体格指数`、`平均血圧`、および`6つの血清測定値`）と、結果変数`y`（ベースラインから1年後の疾患進行の定量的な指標）が含まれています。

|観測数|442|
|----------------------|:---|
|予測変数の数|最初の10列は数値型の予測変数|
|結果/ターゲット|11列目はベースラインから1年後の疾患進行の定量的な指標|
|予測変数の情報|- 年齢（年単位）
||- 性別
||- bmi 体格指数
||- bp 平均血圧
||- s1 tc 総血清コレステロール
||- s2 ldl 低密度リポタンパク質
||- s3 hdl 高密度リポタンパク質
||- s4 tch 総コレステロール / HDL
||- s5 ltg おそらく血清トリグリセリドレベルの対数
||- s6 glu 血糖値|

> 🎓 覚えておいてください、これは教師あり学習であり、'y'という名前のターゲットが必要です。

Rでデータを操作する前に、データをRのメモリにインポートするか、Rがリモートでデータにアクセスできるようにする接続を構築する必要があります。

> [readr](https://readr.tidyverse.org/)パッケージは、Tidyverseの一部であり、Rに長方形データを高速かつ簡単に読み込む方法を提供します。

では、このソースURLで提供されている糖尿病データセットを読み込みましょう: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

また、`glimpse()`を使ってデータの健全性チェックを行い、`slice()`を使って最初の5行を表示します。

さらに進む前に、Rコードで頻繁に遭遇するものを紹介します 🥁🥁: パイプ演算子 `%>%`

パイプ演算子 (`%>%`) は、オブジェクトを関数や呼び出し式に順序立てて渡すことで、論理的な順序で操作を行います。コード内で「そして次に」と言っているようなものだと考えることができます。


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` を使うと、このデータには 442 行と 11 列があり、すべての列がデータ型 `double` であることがわかります。

<br>

> `glimpse()` と `slice()` は [`dplyr`](https://dplyr.tidyverse.org/) の関数です。Dplyr は Tidyverse の一部であり、データ操作の文法を提供します。一貫性のある一連の動詞を通じて、最も一般的なデータ操作の課題を解決するのに役立ちます。

<br>

データが揃ったので、この演習のターゲットとして 1 つの特徴量（`bmi`）に絞り込みましょう。そのためには、必要な列を選択する必要があります。では、どうやってこれを行うのでしょうか？

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) を使うと、データフレーム内の列を*選択*（必要に応じて名前を変更）することができます。


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. トレーニングデータとテストデータ

教師あり学習では、データを2つのサブセットに*分割*することが一般的です。通常は、モデルをトレーニングするための（一般的に大きな）セットと、モデルの性能を確認するための小さな「保持」セットに分けます。

データの準備が整ったので、このデータセット内の数値を論理的に分割する方法を機械が助けてくれるかどうか確認してみましょう。Tidymodelsフレームワークの一部である[rsample](https://tidymodels.github.io/rsample/)パッケージを使用して、データを*どのように*分割するかの情報を含むオブジェクトを作成し、その後、作成されたトレーニングセットとテストセットを抽出するために2つのrsample関数を使用します。


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Tidymodelsで線形回帰モデルをトレーニングする

さあ、モデルをトレーニングする準備が整いました！

Tidymodelsでは、`parsnip()`を使って以下の3つの概念を指定することでモデルを定義します：

-   モデルの**タイプ**は、線形回帰、ロジスティック回帰、決定木モデルなど、モデルの種類を区別します。

-   モデルの**モード**には、回帰や分類といった一般的なオプションが含まれます。一部のモデルタイプはこれらの両方をサポートしていますが、片方のみを持つものもあります。

-   モデルの**エンジン**は、モデルを適合させるために使用される計算ツールです。これには、**`"lm"`** や **`"ranger"`** のようなRパッケージが含まれることが多いです。

これらのモデリング情報はモデル仕様にまとめられます。それでは、モデル仕様を作成してみましょう！


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

モデルが*指定*された後、通常は式とデータを使用して、[`fit()`](https://parsnip.tidymodels.org/reference/fit.html)関数を使ってモデルを`推定`または`訓練`することができます。

`y ~ .`は、`y`を予測される量/ターゲットとしてフィットし、すべての予測因子/特徴量、つまり`.`によって説明されることを意味します（この場合、予測因子は1つだけで、それは`bmi`です）。


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

トレーニング中に学習された係数をモデルの出力から確認できます。これらの係数は、実際の値と予測値の間の全体的な誤差を最小化する最適な直線の係数を表しています。
<br>

## 5. テストセットで予測を行う

モデルをトレーニングしたので、これを使用してテストデータセットの疾患進行 y を予測できます。[parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) を使って予測を行い、データグループ間の線を描画します。


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

やったー！💃🕺 モデルをトレーニングして、予測を行いました！

予測を行う際、tidymodelsの慣例では、標準化された列名を持つ結果のtibble/データフレームを常に生成します。これにより、元のデータと予測結果を使いやすい形式で結合し、プロットなどの後続の操作に活用することが簡単になります。

`dplyr::bind_cols()`は複数のデータフレームを効率的に列として結合します。


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. モデリング結果のプロット

さて、これを視覚的に確認する時間です 📈。テストセットのすべての `y` と `bmi` の値を散布図で描画し、予測値を使ってモデルのデータグループ間で最適な位置に線を引きます。

Rにはグラフを作成するためのいくつかのシステムがありますが、`ggplot2` はその中でも最もエレガントで汎用性の高いものの一つです。これにより、**独立したコンポーネントを組み合わせる**ことでグラフを作成することができます。


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ ここで何が起きているのか少し考えてみましょう。一本の直線が多数の小さなデータ点を通り抜けていますが、具体的に何をしているのでしょうか？この直線を使って、新しい未確認のデータポイントがプロットのy軸に対してどの位置に収まるべきか予測できる方法が見えてきますか？このモデルの実際の用途を言葉で説明してみてください。

おめでとうございます！初めて線形回帰モデルを構築し、それを使って予測を行い、プロットに表示することができました！



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を期すよう努めておりますが、自動翻訳には誤りや不正確な表現が含まれる可能性があります。元の言語で記載された原文を公式な情報源としてご参照ください。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用に起因する誤解や誤認について、当社は一切の責任を負いません。
